In [3]:
import numpy as np
import pandas as pd
import scipy.sparse

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import plotly.graph_objects as go
import pyLDAvis
import pyLDAvis.sklearn
import seaborn as sns
sns.set_style('white')

import nltk

import string
from nltk.corpus import stopwords

import psycopg2 as pg2

import ipywidgets as widgets
from ipywidgets import interact, interactive,fixed

import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import nmf
from sklearn.feature_extraction.text import CountVectorizer


from wordcloud import WordCloud, STOPWORDS
from subprocess import check_output
import matplotlib as mpl

%matplotlib inline

## music lyric look up class

In [2]:
# class that puts all the SQL query actions under the hood.

class Query:

    def total(self,artist):
        conn = pg2.connect(database='Music',user='postgres',password='obscura88')
        cur=conn.cursor()
        cur.execute("SELECT artist, SUM(lyrics_word_length), SUM(cleaned_word_length) FROM song_lyrics WHERE artist = (%s)  GROUP BY artist", [artist])    
        result=cur.fetchone()
        conn.close()   
        return print(f"{result[0]} has {result[1]} total words in their lyrics and {result[2]} excluding stopwords")
        
    def song_count(self,artist):
        conn = pg2.connect(database='Music',user='postgres',password='obscura88')
        cur=conn.cursor()
        cur.execute("SELECT title AS Bon_Iver, lyrics_word_length, cleaned_word_length FROM song_lyrics WHERE artist = (%s)",[artist])
        song=cur.fetchall()
        conn.close() 
        return song
    
    def album_count(self,artist):
        conn = pg2.connect(database='Music',user='postgres',password='obscura88')
        cur=conn.cursor()
        cur.execute("SELECT album, SUM(lyrics_word_length), SUM(cleaned_word_length) FROM song_lyrics WHERE artist = (%s) GROUP BY album",[artist])
        album=cur.fetchall()
        conn.close()
        return album  
        
    def album_avg(self,artist):
        conn = pg2.connect(database='Music',user='postgres',password='obscura88')
        cur=conn.cursor()
        cur.execute("SELECT artist,ROUND(AVG(original),0) AS average_word_count,ROUND(AVG(cleaned),0) AS cleaned_word_count FROM (SELECT artist, album, SUM(lyrics_word_length) AS original, SUM(cleaned_word_length) AS cleaned FROM song_lyrics WHERE artist = (%s) GROUP BY album, artist) AS T GROUP BY artist",[artist])
        albumavg=cur.fetchone()
        conn.close
        print(f'{albumavg[0]} averages {albumavg[1]} words an album {albumavg[2]} of which are non stopwords')
        print('\n')
        return
        
    def song_avg(self,artist):
        conn = pg2.connect(database='Music',user='postgres',password='obscura88')
        cur=conn.cursor()
        cur.execute("SELECT artist, ROUND(AVG(lyrics_word_length),0) AS average_word_count, ROUND(AVG(cleaned_word_length),0) AS average_cleaned_word_count FROM song_lyrics GROUP BY artist ORDER BY average_word_count DESC")
        song_avg=cur.fetchall()
        for j in song_avg:
            if j[0] == artist:
                print(f'{j[0]} has an average of {j[1]} words per song {j[2]} of which are non stopwords')
        print('\n')
        return
    
    def similar(self,artist):
        conn = pg2.connect(database='Music',user='postgres',password='obscura88')
        cur=conn.cursor()
        cur.execute("SELECT artist, rnk FROM ranked_verbose WHERE rnk BETWEEN( SELECT rnk from ranked_verbose WHERE artist = (%s))-3 AND( SELECT rnk from ranked_verbose WHERE artist = (%s))+3",(artist,artist))
        similar=cur.fetchall()
        print(f'Some artists similar to {artist} in terms of verbosity in your library:')
        print('-------------------------------------------------------------------')
        for j in similar:
            if j[0] != artist:
                print(f'{j[0]}')
        print('\n')
        return

    def fluff(self,artist):
        conn = pg2.connect(database='Music',user='postgres',password='obscura88')
        cur=conn.cursor()
        cur.execute("SELECT artist, fluff_ratio FROM fluff_ratio WHERE rnk BETWEEN(SELECT rnk from fluff_ratio WHERE artist = (%s))-3 AND(SELECT rnk from fluff_ratio WHERE artist = (%s))+3",(artist,artist))
        fluff=cur.fetchall()
        for j in fluff:
            if j[0] == artist:
                print(f'Some artists similar to {artist} in terms of fluff in your library:')
                print('(fluff is the ratio of non stopwords divided by all words)')
        print('-------------------------------------------------------------------')
        for j in fluff:
            if j[0] != artist:
                print(f'{j[0]}')
        print('\n')
        return

    
# calls to a set queries from the Query class that and formats it 
def lookup(artist):
    print(f'\nSome info on {artist}!\n')
    print('###########################################################################\n')
    Query().album_avg(artist)
    Query().song_avg(artist)
    Query().similar(artist)
    Query().fluff(artist)

# pulls the list of artists in the SQL library 
def info():

    artists=[]
    conn = pg2.connect(database='Music',user='postgres',password='obscura88')
    cur=conn.cursor()
    cur.execute("SELECT DISTINCT artist FROM song_lyrics ORDER BY artist ASC")
    ar=cur.fetchall()
    for j in ar:
        artists.append(j[0])

    w=interact(lookup, artist=artists)
    display(w)
    
def album_bar(artist):
    
    q=Query().album_count(artist)
    title=[]
    count=[]
    clean=[]
    for j in q:
        title.append(j[0])
        count.append(j[1])
        clean.append(j[2])

    font = {'weight' : 'normal',
            'size'   : 18}
    matplotlib.rc('font', **font)
    x=np.arange(len(title))
    width=.3

    fig,ax = plt.subplots(figsize=(13,10))
    rects1=ax.bar(x-width/2,count,width,label='Word count')
    rects2=ax.bar(x+width/2,clean,width,label='cleaned word count')

    ax.set_xlabel('Album Title',fontsize=16)
    ax.set_ylabel('Word Count',fontsize=16)
    ax.set_title(f'Word count by album for {artist}',fontsize=22)

    ax.set_ylim(0,(max(count)+max(count)*.4))
    ax.set_xticks(x)
    plt.xticks(rotation=90)
    ax.set_xticklabels(title)
    ax.legend(loc='upper right', facecolor='white',framealpha=1)

    ax.yaxis.grid()
    ax.set_axisbelow(True)
    fig.tight_layout()

def info_album():
    
    artists=[]
    conn = pg2.connect(database='Music',user='postgres',password='obscura88')
    cur=conn.cursor()
    cur.execute("SELECT DISTINCT artist FROM song_lyrics ORDER BY artist ASC")
    ar=cur.fetchall()
    for j in ar:
        artists.append(j[0])

    w=interact(album_bar, artist=artists)
    display(w)

    
def song_bar(artist):
    
    q=Query().song_count(artist)
    title=[]
    count=[]
    clean=[]
    for j in q:
        title.append(j[0])
        count.append(j[1])
        clean.append(j[2])

    font = {'weight' : 'normal',
            'size'   : 18}
    matplotlib.rc('font', **font)
    x=np.arange(len(title))
    width=.3

    fig,ax = plt.subplots(figsize=(13,10))
    rects1=ax.bar(x-width/2,count,width,label='Word count')
    rects2=ax.bar(x+width/2,clean,width,label='cleaned word count')

    ax.set_xlabel('Song Title',fontsize=16)
    ax.set_ylabel('Word Count',fontsize=16)
    ax.set_title(f'Word count by song for {artist}',fontsize=22)

    ax.set_ylim(0,(max(count)+max(count)*.4))
    ax.set_xticks(x)
    plt.xticks(rotation=90)
    ax.set_xticklabels(title)
    ax.legend(loc='upper right', facecolor='white',framealpha=1)

    ax.yaxis.grid()
    ax.set_axisbelow(True)
    fig.tight_layout()

def info_song():
    
    artists=[]
    conn = pg2.connect(database='Music',user='postgres',password='obscura88')
    cur=conn.cursor()
    cur.execute("SELECT DISTINCT artist FROM song_lyrics ORDER BY artist ASC")
    ar=cur.fetchall()
    for j in ar:
        artists.append(j[0])

    w=interact(song_bar, artist=artists)
    display(w)


## clean and process

clean the data, input the csv into a SQL database and build a class that outputs info based on artist via SQL queries.

First issue, punctuation. Need to remove punctuation, specifically ' and " from the csv. these characters are used as quote and escape characters by PostgreSQL. Need to remove them from the album, artist, title, and lyrics columns.


In [3]:
songlyrics = pd.read_csv('song_lyrics.csv',index_col='song_id')

In [5]:
def text_process(mess):
    
    return [word for word in mess.split() if word.lower() not in stopwords.words('english')]
    

In [6]:
def no_punc(mess):

    nopunc = [c for c in mess if c not in string.punctuation]
    
    nopunc=''.join(nopunc)
    
    return nopunc
    

In [76]:
def text_punc(mess):
    
    nopunc = [c for c in mess if c not in string.punctuation]
    
    nopunc=''.join(nopunc)
    
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [6]:
songlyrics['artist']=songlyrics['artist'].apply(no_punc)
songlyrics['album']=songlyrics['album'].apply(no_punc)
songlyrics['title']=songlyrics['title'].apply(no_punc)
songlyrics['lyrics']=songlyrics['lyrics'].apply(no_punc)

In [7]:
length=[]
for i,j in songlyrics.iterrows():
    length.append((len(j[7].split())))
songlyrics['lyrics_word_length']=length

In [8]:
songlyrics['cleaned_lyrics']=songlyrics['lyrics'].apply(text_process)

In [9]:
length=[]
for i,j in songlyrics.iterrows():
    length.append(len(j[9]))

songlyrics['cleaned_word_length']=length

In [10]:
songlyrics.head()

,artist,album,title,duration,playcount,rating,rating_interpretation,lyrics,lyrics_word_length,cleaned_lyrics,cleaned_word_length
song_id,,,,,,,,,,,
0,Bon Iver,22 A Million,00000 Million,3:54,1,undefined,never-rated,Mustve been forces that took me on them wild c...,233,"[Mustve, forces, took, wild, courses, knows, m...",121
1,M83,Dead Cities Red Seas Lost Ghosts,0078h,4:02,1,0,not-rated,I was someone elses girl Every time you look i...,179,"[someone, elses, girl, Every, time, look, eyes...",80
2,The Magnetic Fields,50 Song Memoir,02 Be True to Your Bar,3:34,0,undefined,never-rated,Sitting in bars and cafés Writing songs about ...,197,"[Sitting, bars, cafés, Writing, songs, songs, ...",106
3,The Magnetic Fields,50 Song Memoir,05 Never Again,3:19,0,undefined,never-rated,Well walk in the rain Just having a day Never ...,150,"[Well, walk, rain, day, Never, newspaper, page...",75
4,The Magnetic Fields,50 Song Memoir,06 Quotes,2:17,0,undefined,never-rated,For some of which you could have sued Oh the h...,101,"[could, sued, Oh, hateful, mail, youve, read, ...",50


In [11]:
# save cleaned lyrics
songlyrics.to_csv('song_lyrics_cleaned.csv')

removed the punctuation from the artist, album, title, and lyrics columns, removed stopwords from the lyrics and also counted the lyrics and cleaned lyrics and added those columns.

Going to export this file and try to get it into postgres.

Success!

Lets call to it instead of the csv.

Pick an artist,

1) report word count by album, 

2) word count song to song,

3) avgerage would count per song,

4)similar bands based on average word count per song (verbose to mum). 

## query bank

### report artist total word count, original and cleaned

- [x] SELECT artist, SUM(lyrics_word_length), SUM(cleaned_word_length) FROM song_lyrics
WHERE artist = 'Bon Iver'
GROUP BY artist

important to do this by song or by album basis, because some songs would or would not be available in the genius database, so you don't want to unfairly weight the more successful pulls.

### Here is a query that reports the lyric word count on a song by song basis for a given artist

- [x] SELECT title AS Bon_Iver, lyrics_word_length, cleaned_word_length FROM song_lyrics
WHERE artist = 'Bon Iver'


### report artist word count by album orginal and cleaned

- [x] SELECT album, SUM(lyrics_word_length), SUM(cleaned_word_length) FROM song_lyrics
WHERE artist = 'Bon Iver'
GROUP BY album;

### build on this and make a query that finds the average album word count for a given artist 

- [x] SELECT artist,
		ROUND(AVG(original),0) AS average_word_count,
		ROUND(AVG(cleaned),0) AS cleaned_word_count
        FROM
        (
        SELECT artist, album, SUM(lyrics_word_length) AS original, 
 				SUM(cleaned_word_length) AS cleaned
                FROM song_lyrics
                WHERE artist = 'Bon Iver'
                GROUP BY album, artist
                ) AS T
                GROUP BY artist;

### similarly, query that finds the average word count by song for a given artist. This one is ordered by original word count

- [x] SELECT artist, ROUND(AVG(lyrics_word_length),0) AS average_word_count,
			   ROUND(AVG(cleaned_word_length),0) AS average_cleaned_word_count
			   FROM song_lyrics
               GROUP BY artist
               ORDER BY average_word_count DESC

### and this one is ordered by cleaned word count

- [x] SELECT artist, ROUND(AVG(lyrics_word_length),0) AS average_word_count,
			   ROUND(AVG(cleaned_word_length),0) AS average_cleaned_word_count
			   FROM song_lyrics
               GROUP BY artist
               ORDER BY average_cleaned_word_count DESC

### The most challenging idea I had was to return a list of artists that have similar verbosity

SELECT artist
FROM( SELECT 
			   artist, 
			   ROUND(AVG(lyrics_word_length),0) AS average_word_count,
			   ROUND(AVG(cleaned_word_length),0) AS average_cleaned_word_count,
			   ROW_NUMBER() OVER (PARTITION BY 5 ORDER BY ROUND(AVG(lyrics_word_length),0) DESC ) AS rnk
			   FROM song_lyrics

GROUP BY artist)AS T
WHERE rnk BETWEEN(
SELECT rnk FROM 
	(
	SELECT 
			   artist, 
			   ROUND(AVG(lyrics_word_length),0) AS average_word_count,
			   ROUND(AVG(cleaned_word_length),0) AS average_cleaned_word_count,
			   ROW_NUMBER() OVER (PARTITION BY 5 ORDER BY ROUND(AVG(lyrics_word_length),0) DESC ) AS rnk
			   FROM song_lyrics

GROUP BY artist) AS T
WHERE artist = 'Bon Iver')-3 AND(
SELECT rnk FROM
	(
	SELECT 
			   artist, 
			   ROUND(AVG(lyrics_word_length),0) AS average_word_count,
			   ROUND(AVG(cleaned_word_length),0) AS average_cleaned_word_count,
			   ROW_NUMBER() OVER (PARTITION BY 5 ORDER BY ROUND(AVG(lyrics_word_length),0) DESC ) AS rnk
			   FROM song_lyrics

GROUP BY artist) AS T
WHERE artist = 'Bon Iver')+3;

### This is significantly cleaned up by using a view called ranked_verbose

SELECT artist, rnk
FROM ranked_verbose
WHERE rnk 
BETWEEN(
SELECT rnk from ranked_verbose
WHERE artist = 'Bon Iver')-3 
AND(
SELECT rnk from ranked_verbose
WHERE artist = 'Bon Iver')+3

### Similar, a view that makes the ratios of cleaned word count divided by regular word count

SELECT artist, fluff_ratio
FROM fluff_ratio
WHERE rnk 
BETWEEN(
SELECT rnk from fluff_ratio
WHERE artist = 'Bon Iver')-3 
AND(
SELECT rnk from fluff_ratio
WHERE artist = 'Bon Iver')+3

### A lot of the work in this last query was to have a number to pull off of once the group by average lyrics table was created. Headache would be averted if I had an artist_id to use instead of making the rnk column, learned for the future!


#  EDA graphs

Time to make some fun graphs. count the words in each lyrics list

info() will pull info on a band, with comparisons to similar artists.

info_album() will compare lyrics word count with original text and with the stopwords removed for each album of a particular artist.

info_song() will compare lyrics word count with original text and with the stopwords removed for each song of a particular artist.

In [5]:
info()


interactive(children=(Dropdown(description='artist', options=('A Giant Dog', 'A Place To Bury Strangers', 'A P…

<function __main__.lookup(artist)>

## 2) NLP analysis

We can use unsupervised machine learning to further analyze my music library's lyrical content. 

I will apply a topic model to my music library's lyrics. I want to put sets of my songs lyrics into categories differentiated by the words used in the artists songs.  

I found a paper on the topic https://www.researchgate.net/publication/221573745_Natural_language_processing_of_lyrics
and will follow there analysis.

Use a word cloud to get an idea of topics

In [6]:
mpl.rcParams['figure.figsize']=(12.0,12.0)  
mpl.rcParams['font.size']=12            
mpl.rcParams['savefig.dpi']=100             
mpl.rcParams['figure.subplot.bottom']=.1 

wordcloud = WordCloud(
                          background_color='white',
                          max_words=500,
                          max_font_size=40, 
                          random_state=42
                         ).generate(str(songlyrics['cleaned_lyrics'].replace("'","")))

print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

NameError: name 'songlyrics' is not defined

Do a set of -grams graphs. This will pull the most common indiviual words in the lyrics corpus, called unigrams, as well as common two word and three word phrases called bigrams and trigrams.

count vectorizer changes lists of words into vectors, but term frequency-inverse document frequency penalizes the word counts for words that appear very often. They should be less likely to provide insight into the specific topics of a document given how common the word is.

In [66]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(songlyrics['cleaned_lyrics'], 30)
df2 = pd.DataFrame(common_words, columns = ['unigram' , 'count'])

fig = go.Figure([go.Bar(x=df2['unigram'], y=df2['count'])])
fig.update_layout(title=go.layout.Title(text="Top 30 unigrams in the question text after removing stop words and lemmatization"))
fig.show()

In [74]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(songlyrics['cleaned_lyrics'], 30)
df3 = pd.DataFrame(common_words, columns = ['bigram' , 'count'])

fig = go.Figure([go.Bar(x=df3['bigram'], y=df3['count'])])
fig.update_layout(title=go.layout.Title(text="Top 20 bigrams in the question text after removing stop words and lemmatization"))
fig.show()

In [73]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(songlyrics['cleaned_lyrics'], 30)
df4 = pd.DataFrame(common_words, columns = ['trigram' , 'count'])

fig = go.Figure([go.Bar(x=df4['trigram'], y=df4['count'])])
fig.update_layout(title=go.layout.Title(text="Top 30 trigrams in the question text"))
fig.show()

Change the lyrics lists into vectors

In [83]:
#load files at a later date
songlyrics = pd.read_csv('song_lyrics_cleaned.csv',index_col='song_id')
sparse=scipy.sparse.load_npz('sparse_lyrics.npz')

In [110]:
vectorizer = CountVectorizer(analyzer='word',stop_words='english',lowercase=True, token_pattern='[a-zA-Z0-9]{3,}')

bag_lyrics= vectorizer.fit_transform(songlyrics['cleaned_lyrics'])
scipy.sparse.save_npz('sparse_lyrics.npz',bag_lyrics)
LDA_model=LatentDirichletAllocation(n_components=20=,learning_method='online',n_jobs= -1)

LDA_result = LDA_model.fit_transform(bag_lyrics)

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(LDA_model,bag_lyrics,vectorizer)

C:\Users\16142\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14     0.353260 -0.015837       1        1  67.849023
1      0.254042 -0.102332       2        1   4.924372
15     0.152463 -0.115546       3        1   3.358742
18     0.143613  0.016839       4        1   2.703458
4      0.071489  0.000870       5        1   2.245252
9      0.023261  0.153291       6        1   2.194923
16     0.043611  0.307385       7        1   1.912155
17     0.019563 -0.119167       8        1   1.829999
10    -0.011570  0.174102       9        1   1.681653
12    -0.027639 -0.004100      10        1   1.469183
11    -0.055838  0.011325      11        1   1.400890
7     -0.060995 -0.061784      12        1   1.358219
13    -0.074960 -0.022476      13        1   1.257287
2     -0.071706  0.008743      14        1   1.181744
5     -0.114923 -0.003478      15        1   0.991983
6     -0.121242 -0.046076      16        1   0.810405
0     -0.122664 -0.037067      17        1   0.755147
8     -0.135412 -0.045707      18        1   0.705055
3     -0.133420 -0.045046      19        1   0.697225
19    -0.130933 -0.053940      20        1   0.673284, topic_info=      Category          Freq          Term         Total  loglift  logprob
20471  Default   8044.000000          love   8044.000000  30.0000  30.0000
10345  Default  11323.000000          dont  11323.000000  29.0000  29.0000
20003  Default  12050.000000          like  12050.000000  28.0000  28.0000
39464  Default   3133.000000          yeah   3133.000000  27.0000  27.0000
38206  Default   3004.000000         wanna   3004.000000  26.0000  26.0000
20164  Default   2521.000000        little   2521.000000  25.0000  25.0000
2436   Default   3933.000000          away   3933.000000  24.0000  24.0000
12642  Default   2078.000000       feeling   2078.000000  23.0000  23.0000
14726  Default   3146.000000         gonna   3146.000000  22.0000  22.0000
20434  Default   1595.000000          lost   1595.000000  21.0000  21.0000
14718  Default   1693.000000          gone   1693.000000  20.0000  20.0000
14794  Default   6417.000000           got   6417.000000  19.0000  19.0000
16162  Default   1332.000000           hey   1332.000000  18.0000  18.0000
29030  Default   3249.000000         right   3249.000000  17.0000  17.0000
24240  Default   1348.000000           ooh   1348.000000  16.0000  16.0000
19130  Default  10759.000000          know  10759.000000  15.0000  15.0000
33471  Default   1246.000000          stop   1246.000000  14.0000  14.0000
10330  Default   1434.000000           don   1434.000000  13.0000  13.0000
1017   Default   1202.000000          aint   1202.000000  12.0000  12.0000
20931  Default   2023.000000           man   2023.000000  11.0000  11.0000
23341  Default   1404.000000           new   1404.000000  10.0000  10.0000
38209  Default   5900.000000          want   5900.000000   9.0000   9.0000
29530  Default   1581.000000           run   1581.000000   8.0000   8.0000
13883  Default   1018.000000          fuck   1018.000000   7.0000   7.0000
17299  Default    986.000000     important    986.000000   6.0000   6.0000
20182  Default   1716.000000        living   1716.000000   5.0000   5.0000
12264  Default   1684.000000          face   1684.000000   4.0000   4.0000
25489  Default   1241.000000        people   1241.000000   3.0000   3.0000
32410  Default   1190.000000          song   1190.000000   2.0000   2.0000
34005  Default   1495.000000           sun   1495.000000   1.0000   1.0000
...        ...           ...           ...           ...      ...      ...
14421  Topic20     11.892473      gigantic     12.668710   4.9375  -6.2241
10358  Topic20     11.208692     doodoodoo     11.984930   4.9338  -6.2833
37034  Topic20     11.067627       unhappy     11.843865   4.9330  -6.2960
21380  Topic20     11.061240        mayday     11.837502   4.9329  -6.2966
33040  Topic20     10.730800       stacked     11.507038   4.9309  -6

In [111]:
LDA_model=LatentDirichletAllocation(n_components=7,learning_method='online',n_jobs= -1)

LDA_result = LDA_model.fit_transform(bag_lyrics)

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(LDA_model,bag_lyrics,vectorizer)

C:\Users\16142\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.328233 -0.037845       1        1  76.565983
6      0.128164 -0.118824       2        1   6.275809
2      0.009804 -0.126242       3        1   4.952700
5      0.045519  0.349374       4        1   3.492423
3     -0.156225  0.057822       5        1   3.110204
4     -0.155531 -0.063779       6        1   3.082092
0     -0.199963 -0.060505       7        1   2.520790, topic_info=      Category          Freq       Term         Total  loglift  logprob
20003  Default  11192.000000       like  11192.000000  30.0000  30.0000
34909  Default   3518.000000       tell   3518.000000  29.0000  29.0000
20164  Default   2592.000000     little   2592.000000  28.0000  28.0000
36323  Default   2050.000000        try   2050.000000  27.0000  27.0000
15917  Default   2902.000000      heart   2902.000000  26.0000  26.0000
12642  Default   1624.000000    feeling   1624.000000  25.0000  25.0000
16162  Default   1474.000000        hey   1474.000000  24.0000  24.0000
10345  Default  11746.000000       dont  11746.000000  23.0000  23.0000
31010  Default   1201.000000       shes   1201.000000  22.0000  22.0000
20182  Default   1071.000000     living   1071.000000  21.0000  21.0000
19130  Default  11115.000000       know  11115.000000  20.0000  20.0000
2055   Default   1154.000000        ask   1154.000000  19.0000  19.0000
23341  Default   1580.000000        new   1580.000000  18.0000  18.0000
38377  Default   4586.000000        way   4586.000000  17.0000  17.0000
1250   Default   1043.000000    alright   1043.000000  16.0000  16.0000
9694   Default    690.000000       died    690.000000  15.0000  15.0000
15769  Default    725.000000       hate    725.000000  14.0000  14.0000
38122  Default   1336.000000    waiting   1336.000000  13.0000  13.0000
19090  Default    953.000000       knew    953.000000  12.0000  12.0000
12264  Default   1645.000000       face   1645.000000  11.0000  11.0000
38320  Default    922.000000      water    922.000000  10.0000  10.0000
16144  Default    977.000000        hes    977.000000   9.0000   9.0000
17299  Default    607.000000  important    607.000000   8.0000   8.0000
20471  Default   8343.000000       love   8343.000000   7.0000   7.0000
13883  Default    737.000000       fuck    737.000000   6.0000   6.0000
20931  Default   2248.000000        man   2248.000000   5.0000   5.0000
19970  Default   2180.000000      light   2180.000000   4.0000   4.0000
15539  Default   1361.000000       hand   1361.000000   3.0000   3.0000
3470   Default    830.000000        big    830.000000   2.0000   2.0000
39599  Default   7067.000000      youre   7067.000000   1.0000   1.0000
...        ...           ...        ...           ...      ...      ...
28244   Topic7     48.772969  reflektor     49.427218   3.6673  -6.1330
12493   Topic7     48.718135    fashion     49.375273   3.6672  -6.1341
36033   Topic7     48.325628       trap     48.983792   3.6671  -6.1422
7706    Topic7     47.759304     cookie     48.414552   3.6670  -6.1540
4045    Topic7     46.342977       bolt     47.000669   3.6665  -6.1841
15137   Topic7     44.391982      group     45.048882   3.6659  -6.2271
3239    Topic7     43.560592     belief     44.219288   3.6656  -6.2460
39426   Topic7     42.861582     yanada     43.515917   3.6654  -6.2622
20356   Topic7     41.507815        loo     42.162208   3.6650  -6.2943
29660   Topic7     40.786717     safety     41.444642   3.6646  -6.3118
30452   Topic7     40.507989      seeks     41.163262   3.6646  -6.3187
38270   Topic7     39.829334    warrior     40.485227   3.6643  -6.3356
18344   Topic7     37.946242       jean     38.601054   3.6635  -6.3840
342     Topic7     36.741954        aah     37.397990   3.6629  -6.4163
26644   Topic7     36.178669      prawn     36.833624   3.6627  -6.4317
16247   Topic7     36.047792        hih     36.702044   3.6626  -6.4353
38016   Topic7 

In [113]:
LDA_model=LatentDirichletAllocation(n_components=4,learning_method='online',n_jobs= -1)

LDA_result = LDA_model.fit_transform(bag_lyrics)

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(LDA_model,bag_lyrics,vectorizer)

C:\Users\16142\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.248713  0.074624       1        1  73.145277
0     -0.121724 -0.018768       2        1  15.837858
3      0.128591 -0.262985       3        1   6.656216
2      0.241846  0.207128       4        1   4.360649, topic_info=      Category          Freq       Term         Total  loglift  logprob
20003  Default  10831.000000       like  10831.000000  30.0000  30.0000
34909  Default   3413.000000       tell   3413.000000  29.0000  29.0000
39464  Default   3827.000000       yeah   3827.000000  28.0000  28.0000
36323  Default   2000.000000        try   2000.000000  27.0000  27.0000
12642  Default   1610.000000    feeling   1610.000000  26.0000  26.0000
10345  Default  11679.000000       dont  11679.000000  25.0000  25.0000
19130  Default  11046.000000       know  11046.000000  24.0000  24.0000
20182  Default    990.000000     living    990.000000  23.0000  23.0000
15917  Default   2912.000000      heart   2912.000000  22.0000  22.0000
14794  Default   6285.000000        got   6285.000000  21.0000  21.0000
20471  Default   8301.000000       love   8301.000000  20.0000  20.0000
31010  Default   1260.000000       shes   1260.000000  19.0000  19.0000
1250   Default    998.000000    alright    998.000000  18.0000  18.0000
2055   Default    779.000000        ask    779.000000  17.0000  17.0000
38377  Default   4533.000000        way   4533.000000  16.0000  16.0000
35496  Default   6664.000000       time   6664.000000  15.0000  15.0000
38209  Default   6064.000000       want   6064.000000  14.0000  14.0000
20164  Default   2547.000000     little   2547.000000  13.0000  13.0000
17299  Default    528.000000  important    528.000000  12.0000  12.0000
13883  Default    579.000000       fuck    579.000000  11.0000  11.0000
38122  Default   1334.000000    waiting   1334.000000  10.0000  10.0000
3846   Default    860.000000      blood    860.000000   9.0000   9.0000
26914  Default    427.000000   probably    427.000000   8.0000   8.0000
12634  Default   4455.000000       feel   4455.000000   7.0000   7.0000
30756  Default    664.000000        set    664.000000   6.0000   6.0000
22328  Default    548.000000     moment    548.000000   5.0000   5.0000
32386  Default    455.000000   somebody    455.000000   4.0000   4.0000
15839  Default   2158.000000       head   2158.000000   3.0000   3.0000
9694   Default    462.000000       died    462.000000   2.0000   2.0000
17139  Default   4491.000000        ill   4491.000000   1.0000   1.0000
...        ...           ...        ...           ...      ...      ...
33886   Topic4     72.968379      sucks     74.687941   3.1093  -6.2782
8690    Topic4     84.546310     damned     88.004221   3.0925  -6.1309
15074   Topic4     77.333334       grin     79.893420   3.1000  -6.2201
1962    Topic4    246.476739    arrived    305.709217   2.9172  -5.0610
20114   Topic4     72.771709   lipstick     74.996291   3.1024  -6.2809
13883   Topic4    419.690074       fuck    579.997331   2.8090  -4.5287
9465    Topic4     73.334915       desk     76.036215   3.0964  -6.2732
21711   Topic4     93.492100      metal    104.234451   3.0238  -6.0304
9694    Topic4    286.587115       died    462.570502   2.6538  -4.9102
20182   Topic4    470.164013     living    990.252018   2.3877  -4.4151
10696   Topic4    192.467512       drop    295.868493   2.7026  -5.3083
32386   Topic4    255.504196   somebody    455.552128   2.5543  -5.0250
27326   Topic4     92.634848       punk    106.642269   2.9917  -6.0396
34397   Topic4     79.838028     switch     85.602261   3.0628  -6.1882
12642   Topic4    445.166903    feeling   1610.661439   1.8466  -4.4698
22328   Topic4    231.495336     moment    548.175232   2.2705  -5.1237
34909   Topic4    652.104275       tell   3413.699575   1.4772  -4.0880
36326   Topic4     93.183599      tryin    114.398005   2.9274  -6.0337
20003   Topic4   1024.672187     

In [125]:
vectorizer = CountVectorizer(analyzer='word',stop_words='english',max_df=.5, min_df=50, max_features=25000, lowercase=True, token_pattern='[a-zA-Z0-9]{3,}')

bag_lyrics= vectorizer.fit_transform(songlyrics['cleaned_lyrics'])
scipy.sparse.save_npz('sparse_lyrics_2.npz',bag_lyrics)
LDA_model=LatentDirichletAllocation(n_components=6,learning_method='online',n_jobs= -1)

LDA_result = LDA_model.fit_transform(bag_lyrics)

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(LDA_model,bag_lyrics,vectorizer)

C:\Users\16142\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.080267 -0.106587       1        1  29.650592
5     -0.069251  0.128309       2        1  17.747044
2      0.097626  0.181187       3        1  15.767776
1     -0.123227  0.089577       4        1  15.603403
0     -0.142954 -0.188085       5        1  14.263554
3      0.318073 -0.104402       6        1   6.967632, topic_info=     Category          Freq      Term         Total  loglift  logprob
865   Default   6440.000000      love   6440.000000  30.0000  30.0000
382   Default  12044.000000      dont  12044.000000  29.0000  29.0000
776   Default  11379.000000      know  11379.000000  28.0000  28.0000
1461  Default   3849.000000      tell   3849.000000  27.0000  27.0000
1583  Default   3442.000000     wanna   3442.000000  26.0000  26.0000
609   Default   3362.000000     gonna   3362.000000  25.0000  25.0000
827   Default  11832.000000      like  11832.000000  24.0000  24.0000
1697  Default   7235.000000     youre   7235.000000  23.0000  23.0000
57    Default   2641.000000      baby   2641.000000  22.0000  22.0000
612   Default   6700.000000       got   6700.000000  21.0000  21.0000
1584  Default   6216.000000      want   6216.000000  20.0000  20.0000
672   Default   2809.000000     heart   2809.000000  19.0000  19.0000
264   Default   4791.000000      come   4791.000000  18.0000  18.0000
1538  Default   2213.000000       try   2213.000000  17.0000  17.0000
1686  Default   3978.000000      yeah   3978.000000  16.0000  16.0000
53    Default   4108.000000      away   4108.000000  15.0000  15.0000
848   Default   2777.000000      long   2777.000000  14.0000  14.0000
495   Default   2056.000000   feeling   2056.000000  13.0000  13.0000
686   Default   1700.000000       hey   1700.000000  12.0000  12.0000
990   Default   1546.000000       ooh   1546.000000  11.0000  11.0000
1604  Default   4533.000000       way   4533.000000  10.0000  10.0000
381   Default   1268.000000       don   1268.000000   9.0000   9.0000
17    Default   1052.000000   alright   1052.000000   8.0000   8.0000
1477  Default   3480.000000     think   3480.000000   7.0000   7.0000
1142  Default   3212.000000     right   3212.000000   6.0000   6.0000
14    Default   1393.000000      aint   1393.000000   5.0000   5.0000
861   Default   1735.000000      lost   1735.000000   4.0000   4.0000
842   Default   1570.000000    living   1570.000000   3.0000   3.0000
824   Default   2171.000000     light   2171.000000   2.0000   2.0000
608   Default   1815.000000      gone   1815.000000   1.0000   1.0000
...       ...           ...       ...           ...      ...      ...
1074   Topic6    720.638241  probably    920.888174   2.4187  -4.1701
940    Topic6    178.948720    motion    231.235577   2.4076  -5.5631
1465   Topic6    175.704471  terrible    229.066365   2.3987  -5.5814
766    Topic6    256.700499      king    340.891555   2.3802  -5.2023
1461   Topic6   2500.534944      tell   3849.642085   2.2324  -2.9260
930    Topic6    561.717076    moment    799.873490   2.3104  -4.4192
381    Topic6    839.923073       don   1268.223069   2.2518  -4.0169
1538   Topic6   1364.203317       try   2213.824992   2.1797  -3.5319
672    Topic6   1525.079150     heart   2809.255151   2.0530  -3.4204
1319   Topic6    485.568883  somebody    756.292967   2.2208  -4.5649
495    Topic6   1071.425799   feeling   2056.587828   2.0118  -3.7735
660    Topic6    426.728468      hate    678.491875   2.2002  -4.6941
276    Topic6    308.869157   control    498.311550   2.1856  -5.0173
576    Topic6    275.124286    future    432.374218   2.2118  -5.1330
42     Topic6    464.089274       ask   1036.144332   1.8607  -4.6101
1276   Topic6    211.893115    silent    300.074967   2.3159  -5.3941
842    Topic6    585.327865    living   1570.823964   1.6767  -4.3781
773    Topic6    456.151270      knew   1051.343097   1.8289  -4.6274
655    Topic6    501.8107

In [131]:
vectorizer = CountVectorizer(analyzer='word',stop_words='english',max_df=.5, min_df=20, max_features=20000, lowercase=True, token_pattern='[a-zA-Z0-9]{3,}')

bag_lyrics= vectorizer.fit_transform(songlyrics['cleaned_lyrics'])
scipy.sparse.save_npz('sparse_lyrics_2.npz',bag_lyrics)
LDA_model=LatentDirichletAllocation(n_components=5,learning_method='online',n_jobs= -1)

LDA_result = LDA_model.fit_transform(bag_lyrics)

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(LDA_model,bag_lyrics,vectorizer)

C:\Users\16142\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.079392  0.032182       1        1  37.101597
2      0.181264  0.206087       2        1  21.689493
4     -0.217797 -0.008107       3        1  14.957530
0     -0.076287 -0.028577       4        1  14.562754
3      0.192212 -0.201584       5        1  11.688626, topic_info=     Category          Freq       Term         Total  loglift  logprob
849   Default  12119.000000       dont  12119.000000  30.0000  30.0000
1780  Default   8577.000000       love   8577.000000  29.0000  29.0000
1634  Default  11428.000000       know  11428.000000  28.0000  28.0000
3326  Default   6250.000000       want   6250.000000  27.0000  27.0000
1292  Default   6498.000000        got   6498.000000  26.0000  26.0000
3492  Default   4002.000000       yeah   4002.000000  25.0000  25.0000
3325  Default   3452.000000      wanna   3452.000000  24.0000  24.0000
1719  Default  11100.000000       like  11100.000000  23.0000  23.0000
1287  Default   3358.000000      gonna   3358.000000  22.0000  22.0000
3064  Default   3768.000000       tell   3768.000000  21.0000  21.0000
123   Default   2562.000000       baby   2562.000000  20.0000  20.0000
1972  Default   3023.000000       need   3023.000000  19.0000  19.0000
568   Default   4758.000000       come   4758.000000  18.0000  18.0000
2036  Default   1549.000000        ooh   1549.000000  17.0000  17.0000
2412  Default   3449.000000      right   3449.000000  16.0000  16.0000
3505  Default   7242.000000      youre   7242.000000  15.0000  15.0000
1067  Default   1807.000000    feeling   1807.000000  14.0000  14.0000
3221  Default   2157.000000        try   2157.000000  13.0000  13.0000
33    Default   1398.000000       aint   1398.000000  12.0000  12.0000
431   Default   3661.000000      cause   3661.000000  11.0000  11.0000
3134  Default   6828.000000       time   6828.000000  10.0000  10.0000
408   Default   1236.000000       care   1236.000000   9.0000   9.0000
3307  Default   1419.000000       wait   1419.000000   8.0000   8.0000
1527  Default   1009.000000  important   1009.000000   7.0000   7.0000
1556  Default   3789.000000        ive   3789.000000   6.0000   6.0000
1748  Default   1360.000000     living   1360.000000   5.0000   5.0000
2516  Default   4809.000000        say   4809.000000   4.0000   4.0000
1519  Default   4644.000000        ill   4644.000000   3.0000   3.0000
1844  Default    958.000000       mean    958.000000   2.0000   2.0000
130   Default   1394.000000        bad   1394.000000   1.0000   1.0000
...       ...           ...        ...           ...      ...      ...
1748   Topic5   1043.210696     living   1360.032991   1.8813  -4.4296
2004   Topic5    222.025102       nose    239.527284   2.0707  -5.9769
1067   Topic5   1319.567960    feeling   1807.288029   1.8320  -4.1946
896    Topic5    396.251013       drop    461.927404   1.9932  -5.3976
1208   Topic5    768.383477       fuck    990.946508   1.8922  -4.7354
1907   Topic5    563.488036     moment    705.258611   1.9221  -5.0455
2539   Topic5    354.006845  screaming    422.135182   1.9705  -5.5104
2779   Topic5    521.294749   somebody    678.974338   1.8823  -5.1234
2678   Topic5    223.755930     silent    249.361037   2.0382  -5.9691
848    Topic5    754.985205        don   1143.588498   1.7313  -4.7530
94     Topic5    633.846733        ask    921.585831   1.7723  -4.9279
3064   Topic5   1878.759655       tell   3768.700765   1.4504  -3.8413
1598   Topic5    245.507212       kick    280.870061   2.0120  -5.8764
81     Topic5    233.147178        arm    264.049341   2.0221  -5.9280
171    Topic5    362.596400       beat    473.620895   1.8794  -5.4864
1719   Topic5   3807.066290       like  11100.183014   1.0765  -3.1351
704    Topic5    475.288666        cut    705.727085   1.7512  -5.2158
3221   Topic5   1007.694878        try   2157.211385   1.3854  -4.4643
1934   Topic5    409.941077  

In [132]:
LDA_model=LatentDirichletAllocation(n_components=4,learning_method='online',n_jobs= -1)

LDA_result = LDA_model.fit_transform(bag_lyrics)

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(LDA_model,bag_lyrics,vectorizer)

C:\Users\16142\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.095427 -0.003074       1        1  48.151267
1      0.024721 -0.220721       2        1  24.455793
0     -0.181647  0.118231       3        1  17.904100
3      0.252353  0.105564       4        1   9.488841, topic_info=     Category          Freq       Term         Total  loglift  logprob
1780  Default   6214.000000       love   6214.000000  30.0000  30.0000
3492  Default   4041.000000       yeah   4041.000000  29.0000  29.0000
3325  Default   3471.000000      wanna   3471.000000  28.0000  28.0000
1292  Default   6732.000000        got   6732.000000  27.0000  27.0000
3064  Default   3863.000000       tell   3863.000000  26.0000  26.0000
1719  Default  11862.000000       like  11862.000000  25.0000  25.0000
849   Default  11997.000000       dont  11997.000000  24.0000  24.0000
1287  Default   3381.000000      gonna   3381.000000  23.0000  23.0000
1634  Default  11383.000000       know  11383.000000  22.0000  22.0000
3221  Default   2136.000000        try   2136.000000  21.0000  21.0000
1431  Default   1719.000000        hey   1719.000000  20.0000  20.0000
1743  Default   2350.000000     little   2350.000000  19.0000  19.0000
2036  Default   1558.000000        ooh   1558.000000  18.0000  18.0000
1067  Default   2070.000000    feeling   2070.000000  17.0000  17.0000
1527  Default   1110.000000  important   1110.000000  16.0000  16.0000
3326  Default   6222.000000       want   6222.000000  15.0000  15.0000
123   Default   2579.000000       baby   2579.000000  14.0000  14.0000
33    Default   1406.000000       aint   1406.000000  13.0000  13.0000
1819  Default   2318.000000        man   2318.000000  12.0000  12.0000
1748  Default   1520.000000     living   1520.000000  11.0000  11.0000
2618  Default   1352.000000       shes   1352.000000  10.0000  10.0000
1253  Default   1522.000000       girl   1522.000000   9.0000   9.0000
848   Default   1558.000000        don   1558.000000   8.0000   8.0000
3505  Default   7177.000000      youre   7177.000000   7.0000   7.0000
2257  Default    863.000000   probably    863.000000   6.0000   6.0000
1430  Default   1091.000000        hes   1091.000000   5.0000   5.0000
94    Default    952.000000        ask    952.000000   4.0000   4.0000
3134  Default   6788.000000       time   6788.000000   3.0000   3.0000
130   Default   1136.000000        bad   1136.000000   2.0000   2.0000
1519  Default   4586.000000        ill   4586.000000   1.0000   1.0000
...       ...           ...        ...           ...      ...      ...
73     Topic4    143.974260     appear    154.334428   2.2856  -6.2015
3221   Topic4   1345.916263        try   2136.156360   1.8931  -3.9663
3064   Topic4   2166.379844       tell   3863.802051   1.7765  -3.4904
1055   Topic4    232.337067   favorite    273.467904   2.1921  -5.7230
2678   Topic4    227.228757     silent    274.136266   2.1674  -5.7452
1748   Topic4    820.694603     living   1520.031591   1.7387  -4.4610
813    Topic4    242.557231        dip    301.079392   2.1389  -5.6799
1907   Topic4    485.318014     moment    784.131471   1.8753  -4.9864
1743   Topic4   1048.541191     little   2350.361631   1.5479  -4.2160
130    Topic4    622.333197        bad   1136.053725   1.7532  -4.7377
1067   Topic4    917.676651    feeling   2070.285506   1.5415  -4.3493
1783   Topic4    212.771887      lover    262.790477   2.1439  -5.8110
2786   Topic4    487.745315       song   1034.559903   1.6031  -4.9814
1719   Topic4   1973.731777       like  11862.429292   0.5616  -3.5835
801    Topic4    503.217616        die   1161.820301   1.5183  -4.9502
1934   Topic4    336.148298      mouth    655.712569   1.6869  -5.3536
1596   Topic4    208.832132        key    260.014514   2.1358  -5.8296
2687   Topic4    371.385890       sing    889.293609   1.4819  -5.2539
403    Topic4    267.416996    capital    444.098027   1.8478  -5.5824
1681   Topic

In [133]:
def show_topics(vectorizer=vectorizer, lda_model=LDA_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in LDA_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=LDA_model, n_words=20)

df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

#topic 0 gender/biological
#topic 1 elemental
#topic 2 internal
#topic 3 musical

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,got,yeah,wanna,gonna,dont,like,hey,man,baby,know,ooh,want,aint,shes,girl,don,cause,hes,said,boy
Topic 1,like,light,heart,sun,eyes,black,fuck,white,water,night,hand,blood,head,sky,theres,dead,arrived,red,blue,cold
Topic 2,know,dont,time,youre,want,ill,feel,say,way,come,let,ive,away,like,think,right,make,wont,cause,night
Topic 3,love,tell,like,try,important,little,feeling,living,probably,ask,bad,somebody,die,song,moment,drop,lost,bit,sing,alright


In [134]:
vectorizer = CountVectorizer(analyzer='word',stop_words='english',max_df=.4, min_df=20, max_features=22000, lowercase=True, token_pattern='[a-zA-Z0-9]{3,}')

bag_lyrics= vectorizer.fit_transform(songlyrics['cleaned_lyrics'])
scipy.sparse.save_npz('sparse_lyrics_2.npz',bag_lyrics)
LDA_model=LatentDirichletAllocation(n_components=5,learning_method='online',n_jobs= -1)

LDA_result = LDA_model.fit_transform(bag_lyrics)

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(LDA_model,bag_lyrics,vectorizer)

C:\Users\16142\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.038347  0.018519       1        1  28.430912
3     -0.098373  0.024008       2        1  26.339277
2      0.122122 -0.252266       3        1  19.051183
0     -0.224173  0.034995       4        1  16.162186
4      0.238772  0.174745       5        1  10.016441, topic_info=     Category          Freq       Term         Total  loglift  logprob
849   Default  12103.000000       dont  12103.000000  30.0000  30.0000
1634  Default  11431.000000       know  11431.000000  29.0000  29.0000
3326  Default   6267.000000       want   6267.000000  28.0000  28.0000
1719  Default  10511.000000       like  10511.000000  27.0000  27.0000
3325  Default   3444.000000      wanna   3444.000000  26.0000  26.0000
1780  Default   8610.000000       love   8610.000000  25.0000  25.0000
1287  Default   3355.000000      gonna   3355.000000  24.0000  24.0000
3064  Default   3611.000000       tell   3611.000000  23.0000  23.0000
3492  Default   2819.000000       yeah   2819.000000  22.0000  22.0000
1519  Default   4672.000000        ill   4672.000000  21.0000  21.0000
1292  Default   6682.000000        got   6682.000000  20.0000  20.0000
3221  Default   1889.000000        try   1889.000000  19.0000  19.0000
1065  Default   4568.000000       feel   4568.000000  18.0000  18.0000
3134  Default   6836.000000       time   6836.000000  17.0000  17.0000
2036  Default   1546.000000        ooh   1546.000000  16.0000  16.0000
1067  Default   1682.000000    feeling   1682.000000  15.0000  15.0000
1972  Default   2997.000000       need   2997.000000  14.0000  14.0000
3505  Default   7231.000000      youre   7231.000000  13.0000  13.0000
848   Default   1542.000000        don   1542.000000  12.0000  12.0000
1714  Default   2260.000000      light   2260.000000  11.0000  11.0000
1556  Default   3858.000000        ive   3858.000000  10.0000  10.0000
1748  Default   1250.000000     living   1250.000000   9.0000   9.0000
1527  Default    912.000000  important    912.000000   8.0000   8.0000
1431  Default   1687.000000        hey   1687.000000   7.0000   7.0000
1881  Default   2449.000000       mind   2449.000000   6.0000   6.0000
1005  Default   3107.000000       eyes   3107.000000   5.0000   5.0000
3449  Default   3005.000000       wont   3005.000000   4.0000   4.0000
3462  Default   2378.000000      world   2378.000000   3.0000   3.0000
2618  Default   1343.000000       shes   1343.000000   2.0000   2.0000
431   Default   3681.000000      cause   3681.000000   1.0000   1.0000
...       ...           ...        ...           ...      ...      ...
1713   Topic5    142.282709     lifted    145.759849   2.2768  -6.2675
318    Topic5    329.208695  breathing    349.025414   2.2425  -5.4286
1259   Topic5    354.207977     giving    381.977821   2.2255  -5.3554
2091   Topic5    168.805670      parts    175.257538   2.2634  -6.0965
802    Topic5    549.556994       died    645.473863   2.1401  -4.9162
3366   Topic5    133.259353        web    136.549257   2.2766  -6.3330
2539   Topic5    335.976049  screaming    378.981165   2.1805  -5.4082
1748   Topic5    964.601140     living   1250.259444   2.0416  -4.3536
2779   Topic5    505.899237   somebody    606.764633   2.1191  -4.9990
85     Topic5    422.651505    arrived    497.997196   2.1369  -5.1787
1412   Topic5    286.399569     heaven    325.602350   2.1727  -5.5679
1067   Topic5   1195.265204    feeling   1682.616468   1.9590  -4.1392
1208   Topic5    675.944506       fuck    928.842296   1.9831  -4.7092
1892   Topic5    446.279756       miss    573.244817   2.0506  -5.1243
3492   Topic5   1671.086366       yeah   2819.896228   1.7777  -3.8041
3221   Topic5   1158.497905        try   1889.681826   1.8117  -4.1704
1719   Topic5   3835.723341       like  10511.071968   1.2929  -2.9732
3064   Topic5   1683.008788       tell   3611.611928   1.5374  -3.7970
896    Topic5    329.542215  

In [136]:
scipy.sparse.save_npz('sparse_lyrics_2.npz',bag_lyrics)
LDA_model=LatentDirichletAllocation(n_components=3,learning_method='online',n_jobs= -1)

LDA_result = LDA_model.fit_transform(bag_lyrics)

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(LDA_model,bag_lyrics,vectorizer)

C:\Users\16142\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.137567  0.119017       1        1  44.340883
1     -0.095589 -0.134215       2        1  41.140271
0      0.233156  0.015197       3        1  14.518846, topic_info=     Category          Freq       Term         Total  loglift  logprob
849   Default  12209.000000       dont  12209.000000  30.0000  30.0000
1780  Default   8700.000000       love   8700.000000  29.0000  29.0000
1719  Default  11330.000000       like  11330.000000  28.0000  28.0000
1634  Default  11427.000000       know  11427.000000  27.0000  27.0000
1292  Default   5513.000000        got   5513.000000  26.0000  26.0000
3492  Default   2969.000000       yeah   2969.000000  25.0000  25.0000
3326  Default   6288.000000       want   6288.000000  24.0000  24.0000
3064  Default   3871.000000       tell   3871.000000  23.0000  23.0000
3325  Default   3481.000000      wanna   3481.000000  22.0000  22.0000
1067  Default   1937.000000    feeling   1937.000000  21.0000  21.0000
1748  Default   1372.000000     living   1372.000000  20.0000  20.0000
1431  Default   1153.000000        hey   1153.000000  19.0000  19.0000
1527  Default   1043.000000  important   1043.000000  18.0000  18.0000
1208  Default    945.000000       fuck    945.000000  17.0000  17.0000
3221  Default   2321.000000        try   2321.000000  16.0000  16.0000
1287  Default   3040.000000      gonna   3040.000000  15.0000  15.0000
1065  Default   4624.000000       feel   4624.000000  14.0000  14.0000
3449  Default   3019.000000       wont   3019.000000  13.0000  13.0000
2618  Default    939.000000       shes    939.000000  12.0000  12.0000
1972  Default   3023.000000       need   3023.000000  11.0000  11.0000
2257  Default    803.000000   probably    803.000000  10.0000  10.0000
3134  Default   6902.000000       time   6902.000000   9.0000   9.0000
3505  Default   7106.000000      youre   7106.000000   8.0000   8.0000
123   Default   2547.000000       baby   2547.000000   7.0000   7.0000
2036  Default   1567.000000        ooh   1567.000000   6.0000   6.0000
33    Default   1091.000000       aint   1091.000000   5.0000   5.0000
44    Default   1064.000000    alright   1064.000000   4.0000   4.0000
431   Default   3535.000000      cause   3535.000000   3.0000   3.0000
1743  Default   2446.000000     little   2446.000000   2.0000   2.0000
802   Default    717.000000       died    717.000000   1.0000   1.0000
...       ...           ...        ...           ...      ...      ...
2779   Topic3    601.182883   somebody    669.880621   1.8215  -5.1976
2618   Topic3    817.365370       shes    939.251020   1.7907  -4.8904
1748   Topic3   1139.021273     living   1372.030813   1.7436  -4.5586
3492   Topic3   2128.564286       yeah   2969.553781   1.5968  -3.9333
2433   Topic3    330.958952       rock    364.083733   1.8343  -5.7945
2004   Topic3    234.441691       nose    245.482812   1.8837  -6.1393
1438   Topic3    271.831654     higher    296.590136   1.8426  -5.9913
1067   Topic3   1210.182314    feeling   1937.350684   1.4592  -4.4980
1292   Topic3   2749.159506        got   5513.529012   1.2338  -3.6775
842    Topic3    325.709508        dog    372.630314   1.7951  -5.8105
2539   Topic3    364.036705  screaming    437.433196   1.7461  -5.6993
3064   Topic3   1911.642352       tell   3871.296394   1.2241  -4.0408
1430   Topic3    576.203177        hes    800.764936   1.6006  -5.2400
1907   Topic3    545.112161     moment    750.332505   1.6102  -5.2955
1719   Topic3   3891.445092       like  11330.349534   0.8610  -3.3300
94     Topic3    605.027385        ask    974.834567   1.4527  -5.1912
44     Topic3    625.761518    alright   1064.571901   1.3984  -5.1575
33     Topic3    629.275382       aint   1091.313231   1.3792  -5.1519
848    Topic3    623.966404        don   1253.777822   1.2319  -5.1604
1912   Topic3    399.021796      money    554.373527   1.6009  -5.

In [17]:
tfidf = TfidfTransformer().fit(lyrics_bag)

In [18]:
lyrics_tfidf = tfidf.transform(lyrics_bag)

Use non-negative matrix facotrization to pull out the topics